# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [1]:
from torch.cuda.amp import GradScaler
import json
import requests
import random
import string
import secrets
import time
import re
from datetime import datetime
import time
from collections import Counter
import math
import collections
import pickle
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
#from torch.cuda.amp import autocast as autocast_old
from torch.cuda.amp import GradScaler, autocast
#from torch.amp import autocast

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [2]:
CHAR_MAP = {'_': 27, '{': 28, '|': 29}
for i, c in enumerate(string.ascii_lowercase, start=1):
    CHAR_MAP[c] = i

def encode_input(masked_word, original_word, guessed_letters, embedding_len):
    vec = [0] * embedding_len
    missed_vec = [0] * 26
    revealed_mask = [0] * embedding_len
    seq_len = len(masked_word)
    start = embedding_len - seq_len
    mask_pos = []
    for i, ch in enumerate(masked_word):
        idx = CHAR_MAP.get(ch, 0)
        vec[start + i] = idx
        if ch == '_':
            mask_pos.append(start + i)
        elif ch in string.ascii_lowercase:
            revealed_mask[start + i] = 1
    for g in guessed_letters:
        if g in string.ascii_lowercase and g not in original_word:
            missed_vec[ord(g) - ord('a')] = 1
    return vec, missed_vec, revealed_mask, mask_pos


class HangmanDataset(Dataset):
    def __init__(self, words, max_len=35, embedding_len=40):
        self.words = [w for w in words if w.isalpha()]
        self.max_len = max_len
        self.embedding_len = embedding_len

    def __len__(self):
        return len(self.words)

    def __getitem__(self, idx):
        raw = self.words[idx]
        padded = '{' + raw + '|'
        reveal_percent = random.choice([0, 0.2, 0.5, 0.8])
        num_reveals = int(len(raw) * reveal_percent)
        reveal_positions = (
            random.sample(range(1, len(padded) - 1), num_reveals)
            if num_reveals > 0
            else []
        )
        masked = list(padded)
        guessed_letters = [padded[pos] for pos in reveal_positions]
        num_masks = random.randint(1, max(1, len(raw) - num_reveals))
        mask_positions = random.sample(
            [i for i in range(1, len(padded) - 1) if i not in reveal_positions], num_masks
        )
        for pos in mask_positions:
            masked[pos] = '_'

        inp_vec, missed_vec, revealed_mask, mask_pos = encode_input(
            ''.join(masked), raw, guessed_letters, self.embedding_len
        )

        if not mask_pos:
            target_pos = 0
            y_letter = 0
        else:
            target_pos = random.choice(mask_pos)
            start = self.embedding_len - len(padded)
            target_idx = target_pos - start
            target_letter = padded[target_idx]
            y_letter = ord(target_letter) - ord('a')

        return (
            torch.tensor(inp_vec, dtype=torch.long),
            torch.tensor(missed_vec, dtype=torch.long),
            torch.tensor(revealed_mask, dtype=torch.long),
            torch.tensor(y_letter, dtype=torch.long),
            torch.tensor(target_pos, dtype=torch.long),
        )


def compute_pos_bias(words, embedding_len):
    counts = np.zeros((26, embedding_len), dtype=float)
    for w in words:
        padded = '{' + w + '|'
        start = embedding_len - len(padded)
        for i, ch in enumerate(padded):
            if 'a' <= ch <= 'z':
                counts[ord(ch) - 97, start + i] += 1
    bias = counts / (counts.sum(axis=0, keepdims=True) + 1e-9)
    return torch.tensor(bias, dtype=torch.float32)


class HangmanLSTM(nn.Module):
    def __init__(self, vocab_size, emb_dim, missed_dim, lstm_hidden_dim, num_layers, dropout_p=0.5):
        super().__init__()
        self.char_embed = nn.Embedding(vocab_size, emb_dim)

        self.missed_embed = nn.Linear(26, missed_dim)

        self.lstm = nn.LSTM(
            emb_dim + missed_dim,
            lstm_hidden_dim,
            num_layers,
            batch_first=True,
            bidirectional=True,
        )

        self.dropout = nn.Dropout(p=dropout_p)

        self.fc = nn.Linear(lstm_hidden_dim * 2, 26)

    def forward(self, x, missed):
        b, seq = x.size()
        emb = self.char_embed(x)  
        missed_emb = self.missed_embed(missed.float())  
        missed_emb_exp = missed_emb.unsqueeze(1).expand(-1, seq, -1)  
        inp = torch.cat([emb, missed_emb_exp], dim=-1) 

        lstm_out, _ = self.lstm(inp)  

        
        lstm_out = self.dropout(lstm_out)  

        logits = self.fc(lstm_out)  
        return logits


with open('/kaggle/input/words-250000-train-txt/words_250000_train.txt') as f:
    all_words = f.read().splitlines()

random.shuffle(all_words)
split_idx = int(0.95 * len(all_words))
train_words, val_words = all_words[:split_idx], all_words[split_idx:]


embedding_len = max(len(w) for w in all_words) + 2

vocab_size = len(CHAR_MAP) + 1  # +1 for zero‐padding index
train_ds = HangmanDataset(train_words, embedding_len=embedding_len)
val_ds   = HangmanDataset(val_words,   embedding_len=embedding_len)

dl_train = DataLoader(train_ds, batch_size=256, shuffle=True,  num_workers=4)
dl_val   = DataLoader(val_ds,   batch_size=256, shuffle=False, num_workers=4)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


new_hidden_dim = 512

model = HangmanLSTM(
    vocab_size=vocab_size,
    emb_dim=64,            
    missed_dim=16,         
    lstm_hidden_dim=new_hidden_dim,
    num_layers=2,          
    dropout_p=0.5          
).to(device)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-2)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)

pos_bias = compute_pos_bias(train_words, embedding_len).to(device)
scaler = GradScaler()

epochs = 18
lambda_pos = 0.2
best_win_rate = 0.0
patience = 5
no_improve = 0

for ep in range(1, epochs + 1):
    model.train()
    total_loss = 0
    start_time = time.time()

    for X, missed, revealed, y_letter, pos in dl_train:
        X, missed, revealed, y_letter, pos = [t.to(device) for t in (X, missed, revealed, y_letter, pos)]
        optimizer.zero_grad()
        with autocast():
            logits = model(X, missed)
          
            L = logits[torch.arange(X.size(0)), pos]
           
            L = L + lambda_pos * torch.log(pos_bias[:, pos].transpose(0, 1) + 1e-9)
            loss = criterion(L, y_letter)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item()

    scheduler.step()
    epoch_time = time.time() - start_time
    avg_loss = total_loss / len(dl_train)
    print(f"Epoch {ep}/{epochs} | Loss: {avg_loss:.4f} | Time: {epoch_time:.2f} sec")

   
    model.eval()
    wins = 0
    total_games = 0

    with torch.no_grad():
        for word in val_words:
            guessed_letters = []
            current_state = ['_'] * len(word)
            incorrect_guesses = 0

            while '_' in current_state and incorrect_guesses < 6:
                clean = ''.join(current_state)
                VOWEL_ORDER = ['e', 'a', 'o', 'i', 'u']

                if (not any(v in clean for v in VOWEL_ORDER)) and (not any(g in guessed_letters for g in VOWEL_ORDER)):
                    guess = next((v for v in VOWEL_ORDER if v not in guessed_letters), None)

                else:
                    pattern = ''.join(c if c != '_' else '.' for c in clean)
                    possible_words = [
                        w for w in val_words
                        if len(w) == len(clean) and re.match(f"^{pattern}$", w)
                    ]
                    candidate_letters = [c for c in string.ascii_lowercase if c not in guessed_letters]

                    if possible_words and candidate_letters:
                        letter_counts = {
                            c: sum(1 for w in possible_words if c in w) for c in candidate_letters
                        }
                        max_count = max(letter_counts.values())
                        best_letters = [c for c, ct in letter_counts.items() if ct == max_count]

                        if len(best_letters) == 1:
                            guess = best_letters[0]
                        else:
                            padded = '{' + clean + '|'
                            vec, missed_vec, revealed_mask, _ = encode_input(padded, word, guessed_letters, embedding_len)
                            X_ = torch.tensor([vec], dtype=torch.long).to(device)
                            missed_ = torch.tensor([missed_vec], dtype=torch.long).to(device)
                            logits = model(X_, missed_)[0]  # (seq, 26)
                            probs = torch.softmax(logits, dim=-1)

                            start = embedding_len - (len(clean) + 2)
                            unrevealed_j = [j for j in range(len(clean)) if clean[j] == '_']
                            unrevealed_emb_positions = [start + 1 + j for j in unrevealed_j]

                            if unrevealed_emb_positions:
                                probs_unrevealed = probs[unrevealed_emb_positions]  # (num_blanks, 26)
                                total_probs = probs_unrevealed.sum(dim=0)
                                masked_total_probs = total_probs.clone()
                                for g in guessed_letters:
                                    idx = ord(g) - 97
                                    if 0 <= idx < 26:
                                        masked_total_probs[idx] = 0
                                guess = chr(torch.argmax(masked_total_probs).item() + 97)
                            else:
                                guess = 'a'

                    else:
                      
                        padded = '{' + clean + '|'
                        vec, missed_vec, revealed_mask, _ = encode_input(padded, word, guessed_letters, embedding_len)
                        X_ = torch.tensor([vec], dtype=torch.long).to(device)
                        missed_ = torch.tensor([missed_vec], dtype=torch.long).to(device)
                        logits = model(X_, missed_)[0]  # (seq, 26)
                        probs = torch.softmax(logits, dim=-1)

                        start = embedding_len - (len(clean) + 2)
                        unrevealed_j = [j for j in range(len(clean)) if clean[j] == '_']
                        unrevealed_emb_positions = [start + 1 + j for j in unrevealed_j]

                        if unrevealed_emb_positions:
                            probs_unrevealed = probs[unrevealed_emb_positions]  # (num_blanks, 26)
                            total_probs = probs_unrevealed.sum(dim=0)
                            masked_total_probs = total_probs.clone()
                            for g in guessed_letters:
                                idx = ord(g) - 97
                                if 0 <= idx < 26:
                                    masked_total_probs[idx] = 0
                            guess = chr(torch.argmax(masked_total_probs).item() + 97)
                        else:
                            guess = 'a'

                guessed_letters.append(guess)
                if guess in word:
                    for i, c in enumerate(word):
                        if c == guess:
                            current_state[i] = c
                else:
                    incorrect_guesses += 1

            if '_' not in current_state:
                wins += 1
            total_games += 1

    win_rate = wins / total_games * 100
    print(f"Val Win Rate: {win_rate:.2f}% | Best: {best_win_rate:.2f}%")

    if win_rate > best_win_rate:
        best_win_rate = win_rate
        torch.save(model.state_dict(), 'best_hangman_lstm.pt')
        no_improve = 0
    else:
        no_improve += 1
        if no_improve >= patience:
            print(f"No improvement over {patience} epochs. Stopping early.")
            break


/tmp/ipykernel_35/212238138.py:160: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/tmp/ipykernel_35/212238138.py:176: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1/18 | Loss: 2.5105 | Time: 64.18 sec
Val Win Rate: 65.83% | Best: 0.00%
Epoch 2/18 | Loss: 2.3182 | Time: 63.53 sec
Val Win Rate: 69.53% | Best: 65.83%
Epoch 3/18 | Loss: 2.2563 | Time: 63.78 sec
Val Win Rate: 70.88% | Best: 69.53%
Epoch 4/18 | Loss: 2.2121 | Time: 63.89 sec
Val Win Rate: 71.86% | Best: 70.88%
Epoch 5/18 | Loss: 2.1873 | Time: 63.74 sec
Val Win Rate: 72.68% | Best: 71.86%
Epoch 6/18 | Loss: 2.1658 | Time: 63.81 sec
Val Win Rate: 72.96% | Best: 72.68%
Epoch 7/18 | Loss: 2.1478 | Time: 63.67 sec
Val Win Rate: 72.47% | Best: 72.96%
Epoch 8/18 | Loss: 2.1328 | Time: 63.91 sec
Val Win Rate: 73.39% | Best: 72.96%
Epoch 9/18 | Loss: 2.1185 | Time: 63.67 sec
Val Win Rate: 74.42% | Best: 73.39%
Epoch 10/18 | Loss: 2.1052 | Time: 63.61 sec
Val Win Rate: 73.49% | Best: 74.42%
Epoch 11/18 | Loss: 2.0924 | Time: 63.55 sec
Val Win Rate: 73.95% | Best: 74.42%
Epoch 12/18 | Loss: 2.0869 | Time: 63.94 sec
Val Win Rate: 73.86% | Best: 74.42%
Epoch 13/18 | Loss: 2.0748 | Time: 63.

In [14]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None, device=None):
        self.hangman_url = 'https://trexsim.com/trexsim/hangman'
        self.session = session or __import__('requests').Session()
        self.access_token = access_token
        self.timeout = timeout
        self.device = device or torch.device('cpu')

        with open('/kaggle/input/words-250000-train-txt/words_250000_train.txt') as f:
            self.full_dictionary = f.read().splitlines()

        self.embedding_len = max(len(w) for w in self.full_dictionary) + 2
        self.pos_bias = compute_pos_bias(self.full_dictionary, self.embedding_len).to(self.device)
        self.model = model.to(self.device)
        self.model.eval()

    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com']
        data = {link: 0 for link in links}
        for link in links:
            requests.get(link, verify=False)
            for _ in range(10):
                s = time.time()
                requests.get(link, verify=False)
                data[link] = time.time() - s
        best = min(data, key=data.get)
        return best + '/trexsim/hangman'

    def guess(self, word):
        clean = word.replace(' ', '')
        VOWEL_ORDER = ['e', 'a', 'o', 'i', 'u']
        alpha = 0.6

        if (not any(v in clean for v in VOWEL_ORDER)) and (not any(g in self.guessed_letters for g in VOWEL_ORDER)):
            for v in VOWEL_ORDER:
                if v not in self.guessed_letters:
                    self.guessed_letters.append(v)
                    return v

        L = len(clean)
        pattern = ''.join(c if c != '_' else '.' for c in clean)
        possible_words = [
            w for w in self.full_dictionary
            if len(w) == L and re.match(f"^{pattern}$", w)
        ]
        candidate_letters = [c for c in string.ascii_lowercase if c not in self.guessed_letters]

        if possible_words and candidate_letters:
            letter_counts = {
                c: sum(1 for w in possible_words if c in w) for c in candidate_letters
            }
            max_count = max(letter_counts.values()) if letter_counts else 0
            if max_count > 0:
                freq_scores = {c: count / max_count for c, count in letter_counts.items()}
            else:
                freq_scores = {c: 0.0 for c in candidate_letters}
        else:
            freq_scores = {c: 0.0 for c in candidate_letters}

        padded = '{' + clean + '|'
        vec, missed_vec, revealed_mask, unrevealed_emb_positions = encode_input(
            padded, clean, self.guessed_letters, self.embedding_len
        )
        X = torch.tensor([vec], dtype=torch.long).to(self.device)
        missed = torch.tensor([missed_vec], dtype=torch.long).to(self.device)
        with torch.no_grad():
            logits = self.model(X, missed)[0]
            probs = torch.softmax(logits, dim=-1)

            if unrevealed_emb_positions:
                probs_unrevealed = probs[unrevealed_emb_positions]
                total_probs = probs_unrevealed.sum(dim=0)
                masked_total_probs = total_probs.clone()
                for g in self.guessed_letters:
                    idx = ord(g) - 97
                    if 0 <= idx < 26:
                        masked_total_probs[idx] = 0
                model_scores = {chr(i + 97): prob.item() for i, prob in enumerate(masked_total_probs)}
            else:
                model_scores = {c: 0.0 for c in candidate_letters}

        combined_scores = {}
        for c in candidate_letters:
            model_score = model_scores.get(c, 0.0)
            freq_score = freq_scores.get(c, 0.0)
            combined_scores[c] = alpha * model_score + (1 - alpha) * freq_score

        if combined_scores:
            chosen_letter = max(combined_scores, key=combined_scores.get)
        else:
            chosen_letter = 'a'  # Fallback

        self.guessed_letters.append(chosen_letter)
        return chosen_letter

    def build_dictionary(self, dictionary_file_location):
        with open(dictionary_file_location, "r") as f:
            return f.read().splitlines()

    def start_game(self, practice=True, verbose=True):
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
        response = self.request("/new_game", {"practice": practice})
        if response.get('status') == "approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print(f"Successfully start a new game! Game ID: {game_id}. # of tries remaining: {tries_remains}. Word: {word}.")
            while tries_remains > 0:
                guess_letter = self.guess(word)
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print(f"Guessing letter: {guess_letter}")
                try:
                    res = self.request("/guess_letter", {"request": "guess_letter", "game_id": game_id, "letter": guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
                if verbose:
                    print(f"Server response: {res}")
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status == "success":
                    if verbose:
                        print(f"Successfully finished game: {game_id}")
                    return True
                elif status == "failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print(f"Failed game: {game_id}. Because of: {reason}")
                    return False
                elif status == "ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status == "success"

    def my_status(self):
        return self.request("/my_status", {})

    def request(self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"
        if self.access_token:
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token
        time.sleep(0.2)
        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)
        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')
        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result

class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""
        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result
        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [15]:
api = HangmanAPI(access_token="77aa615885a84b671a77576d14562a", timeout=2000)

## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [13]:
for i in range(100):
    api.start_game(practice=1,verbose=False)
    [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
    practice_success_rate = total_practice_successes / total_practice_runs
    print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))

run 386 practice games out of an allotted 100,000. practice success rate so far = 0.541
run 387 practice games out of an allotted 100,000. practice success rate so far = 0.543
run 388 practice games out of an allotted 100,000. practice success rate so far = 0.544
run 389 practice games out of an allotted 100,000. practice success rate so far = 0.542
run 390 practice games out of an allotted 100,000. practice success rate so far = 0.544
run 391 practice games out of an allotted 100,000. practice success rate so far = 0.545
run 392 practice games out of an allotted 100,000. practice success rate so far = 0.546
run 393 practice games out of an allotted 100,000. practice success rate so far = 0.547
run 394 practice games out of an allotted 100,000. practice success rate so far = 0.548
run 395 practice games out of an allotted 100,000. practice success rate so far = 0.547
run 396 practice games out of an allotted 100,000. practice success rate so far = 0.548
run 397 practice games out of an

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [16]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=0,verbose=False)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

Playing  0  th game
Playing  1  th game
Playing  2  th game
Playing  3  th game
Playing  4  th game
Playing  5  th game
Playing  6  th game
Playing  7  th game
Playing  8  th game
Playing  9  th game
Playing  10  th game
Playing  11  th game
Playing  12  th game
Playing  13  th game
Playing  14  th game
Playing  15  th game
Playing  16  th game
Playing  17  th game
Playing  18  th game
Playing  19  th game
Playing  20  th game
Playing  21  th game
Playing  22  th game
Playing  23  th game
Playing  24  th game
Playing  25  th game
Playing  26  th game
Playing  27  th game
Playing  28  th game
Playing  29  th game
Playing  30  th game
Playing  31  th game
Playing  32  th game
Playing  33  th game
Playing  34  th game
Playing  35  th game
Playing  36  th game
Playing  37  th game
Playing  38  th game
Playing  39  th game
Playing  40  th game
Playing  41  th game
Playing  42  th game
Playing  43  th game
Playing  44  th game
Playing  45  th game
Playing  46  th game
Playing  47  th game
Pl

HangmanAPIError: {'error': 'You have reached 1000 of games', 'status': 'denied'}

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [17]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

overall success rate = 0.576
